In [ ]:
# =============================================================================
# PHÂN TÍCH DỮ LIỆU CHỨNG KHOÁN VỚI PYSPARK
# Stock Price Big Data Analysis
# =============================================================================

# %% [markdown]
# ## 1. Import thư viện và khởi tạo Spark Session

# %%
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Thiết lập style cho matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Đã import các thư viện thành công")

# %%
# Tạo Spark Session kết nối với Spark Master
spark = SparkSession.builder \
    .appName("Stock Price Big Data Analysis") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "2g") \
    .config("spark.executor.cores", "2") \
    .config("spark.cores.max", "8") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .getOrCreate()

print("✅ Spark Session đã được khởi tạo")
print(f"📊 Spark Version: {spark.version}")
print(f"🎯 Application ID: {spark.sparkContext.applicationId}")
print(f"🔗 Master: {spark.sparkContext.master}")

# %% [markdown]
# ## 2. Đọc dữ liệu từ HDFS

# %%
# Định nghĩa schema cho dữ liệu
stock_schema = StructType([
    StructField("Date", StringType(), True),
    StructField("Open", DoubleType(), True),
    StructField("High", DoubleType(), True),
    StructField("Low", DoubleType(), True),
    StructField("Close", DoubleType(), True),
    StructField("Volume", LongType(), True)
])

# Đọc tất cả file CSV từ HDFS
hdfs_path = "hdfs://namenode:9000/datack/*.csv"
print(f"📂 Đọc dữ liệu từ: {hdfs_path}")

df = spark.read \
    .option("header", "true") \
    .schema(stock_schema) \
    .csv(hdfs_path)

# Thêm cột Symbol từ tên file
df = df.withColumn("filename", input_file_name())
df = df.withColumn("Symbol", regexp_extract(col("filename"), r"data-(\w+)_", 1))

# Chuyển đổi Date sang timestamp
df = df.withColumn("Date", to_date(col("Date"), "yyyy-MM-dd"))

# Cache data để tăng tốc độ xử lý
df.cache()

print(f"\n✅ Đã đọc thành công {df.count():,} dòng dữ liệu")
print(f"📊 Số lượng cổ phiếu: {df.select('Symbol').distinct().count()}")

# %%
# Xem cấu trúc dữ liệu
print("\n📋 CẤU TRÚC DỮ LIỆU:")
print("=" * 80)
df.printSchema()

print("\n📋 DỮ LIỆU MẪU:")
print("=" * 80)
df.show(10)

# %% [markdown]
# ## 3. Thống kê mô tả

# %%
print("\n📊 THỐNG KÊ MÔ TẢ")
print("=" * 80)
df.select("Open", "High", "Low", "Close", "Volume").describe().show()

# %%
# Phạm vi thời gian dữ liệu
date_range = df.agg(
    min("Date").alias("Start_Date"),
    max("Date").alias("End_Date")
).collect()[0]

print(f"\n📅 Phạm vi thời gian:")
print(f"   Từ: {date_range['Start_Date']}")
print(f"   Đến: {date_range['End_Date']}")

# %% [markdown]
# ## 4. Phân tích biến động giá

# %%
# Tính toán các chỉ số kỹ thuật
df_analysis = df.withColumn("Daily_Return", (col("Close") - col("Open")) / col("Open") * 100)
df_analysis = df_analysis.withColumn("Price_Range", col("High") - col("Low"))
df_analysis = df_analysis.withColumn("Volatility", (col("High") - col("Low")) / col("Open") * 100)

# Tính Moving Average 7 ngày và 30 ngày
window_7 = Window.partitionBy("Symbol").orderBy("Date").rowsBetween(-6, 0)
window_30 = Window.partitionBy("Symbol").orderBy("Date").rowsBetween(-29, 0)

df_analysis = df_analysis.withColumn("MA_7", avg("Close").over(window_7))
df_analysis = df_analysis.withColumn("MA_30", avg("Close").over(window_30))

print("\n✅ Đã tính toán xong các chỉ số kỹ thuật")
print("\n📊 DỮ LIỆU SAU KHI PHÂN TÍCH:")
print("=" * 80)
df_analysis.select("Symbol", "Date", "Close", "Daily_Return", "MA_7", "MA_30").show(10)

# %%
# TOP 10 cổ phiếu có giá đóng cửa cao nhất
print("\n💰 TOP 10 CỔ PHIẾU GIÁ CAO NHẤT")
print("=" * 80)
top_expensive = df.groupBy("Symbol") \
    .agg(max("Close").alias("Max_Price")) \
    .orderBy(desc("Max_Price")) \
    .limit(10)
top_expensive.show()

# %%
# TOP 10 cổ phiếu có khối lượng giao dịch cao nhất
print("\n📈 TOP 10 CỔ PHIẾU KHỐI LƯỢNG GIAO DỊCH CAO NHẤT")
print("=" * 80)
top_volume = df.groupBy("Symbol") \
    .agg(sum("Volume").alias("Total_Volume")) \
    .orderBy(desc("Total_Volume")) \
    .limit(10)
top_volume.show()

# %% [markdown]
# ## 5. Phân tích xu hướng thị trường

# %%
# Tính tổng giá trị giao dịch theo năm
df_yearly = df.withColumn("Year", year("Date")) \
    .withColumn("Trade_Value", col("Close") * col("Volume")) \
    .groupBy("Year") \
    .agg(
        sum("Trade_Value").alias("Total_Trade_Value"),
        sum("Volume").alias("Total_Volume"),
        avg("Close").alias("Avg_Price")
    ) \
    .orderBy("Year")

print("\n📊 PHÂN TÍCH THEO NĂM")
print("=" * 80)
df_yearly.show()

# %%
# Trực quan hóa xu hướng theo năm
yearly_pd = df_yearly.toPandas()

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('PHÂN TÍCH XU HƯỚNG THỊ TRƯỜNG THEO NĂM', fontsize=16, fontweight='bold')

# Giá trị giao dịch
axes[0, 0].plot(yearly_pd['Year'], yearly_pd['Total_Trade_Value'], marker='o', linewidth=2)
axes[0, 0].set_title('Tổng Giá Trị Giao Dịch', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Năm')
axes[0, 0].set_ylabel('Giá trị (USD)')
axes[0, 0].grid(True, alpha=0.3)

# Khối lượng giao dịch
axes[0, 1].plot(yearly_pd['Year'], yearly_pd['Total_Volume'], marker='s', color='green', linewidth=2)
axes[0, 1].set_title('Tổng Khối Lượng Giao Dịch', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Năm')
axes[0, 1].set_ylabel('Khối lượng')
axes[0, 1].grid(True, alpha=0.3)

# Giá trung bình
axes[1, 0].plot(yearly_pd['Year'], yearly_pd['Avg_Price'], marker='^', color='orange', linewidth=2)
axes[1, 0].set_title('Giá Trung Bình', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Năm')
axes[1, 0].set_ylabel('Giá (USD)')
axes[1, 0].grid(True, alpha=0.3)

# Biểu đồ tổng hợp
axes[1, 1].bar(yearly_pd['Year'], yearly_pd['Total_Trade_Value'], alpha=0.7)
axes[1, 1].set_title('Biểu Đồ Cột Giá Trị Giao Dịch', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Năm')
axes[1, 1].set_ylabel('Giá trị (USD)')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("✅ Đã vẽ biểu đồ phân tích theo năm")

# %% [markdown]
# ## 6. Phân tích cổ phiếu cụ thể

# %%
# Chọn một số cổ phiếu nổi tiếng để phân tích chi tiết
selected_stocks = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA']

# Lọc dữ liệu
df_selected = df_analysis.filter(col("Symbol").isin(selected_stocks)) \
    .orderBy("Symbol", "Date")

print(f"\n📌 Phân tích {len(selected_stocks)} cổ phiếu: {', '.join(selected_stocks)}")
print(f"📊 Tổng số dòng dữ liệu: {df_selected.count():,}")

# %%
# Chuyển sang Pandas để vẽ biểu đồ
df_selected_pd = df_selected.select("Symbol", "Date", "Close", "MA_7", "MA_30").toPandas()

# Vẽ biểu đồ giá cổ phiếu
fig, axes = plt.subplots(len(selected_stocks), 1, figsize=(16, 4*len(selected_stocks)))
fig.suptitle('BIỂU ĐỒ GIÁ CỔ PHIẾU VÀ MOVING AVERAGE', fontsize=16, fontweight='bold')

for idx, stock in enumerate(selected_stocks):
    stock_data = df_selected_pd[df_selected_pd['Symbol'] == stock].sort_values('Date')
    
    if len(stock_data) > 0:
        ax = axes[idx] if len(selected_stocks) > 1 else axes
        
        ax.plot(stock_data['Date'], stock_data['Close'], label='Close Price', linewidth=1.5, alpha=0.8)
        ax.plot(stock_data['Date'], stock_data['MA_7'], label='MA 7', linewidth=1, alpha=0.7)
        ax.plot(stock_data['Date'], stock_data['MA_30'], label='MA 30', linewidth=1, alpha=0.7)
        
        ax.set_title(f'{stock} - Giá Đóng Cửa và Moving Average', fontsize=12, fontweight='bold')
        ax.set_xlabel('Ngày')
        ax.set_ylabel('Giá (USD)')
        ax.legend()
        ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Đã vẽ biểu đồ giá cổ phiếu")

# %% [markdown]
# ## 7. Phân tích tương quan

# %%
# Tính ma trận tương quan giữa các cổ phiếu
# Pivot data để có giá đóng cửa của từng cổ phiếu theo ngày
df_pivot = df_selected.groupBy("Date").pivot("Symbol").agg(first("Close"))

# Chuyển sang Pandas để tính correlation
df_corr_pd = df_pivot.toPandas().set_index('Date')
correlation_matrix = df_corr_pd.corr()

print("\n🔗 MA TRẬN TƯƠNG QUAN GIỮA CÁC CỔ PHIẾU")
print("=" * 80)
print(correlation_matrix)

# %%
# Vẽ heatmap tương quan
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('MA TRẬN TƯƠNG QUAN GIỮA CÁC CỔ PHIẾU', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("✅ Đã vẽ heatmap tương quan")

# %% [markdown]
# ## 8. Phân tích rủi ro và lợi nhuận

# %%
# Tính toán chỉ số rủi ro và lợi nhuận
risk_return = df_analysis.groupBy("Symbol").agg(
    avg("Daily_Return").alias("Avg_Return"),
    stddev("Daily_Return").alias("Risk_StdDev"),
    min("Daily_Return").alias("Min_Return"),
    max("Daily_Return").alias("Max_Return")
).orderBy(desc("Avg_Return"))

print("\n⚖️ PHÂN TÍCH RỦI RO VÀ LỢI NHUẬN")
print("=" * 80)
risk_return.show(20)

# %%
# Biểu đồ Risk-Return
risk_return_pd = risk_return.toPandas()

plt.figure(figsize=(12, 8))
plt.scatter(risk_return_pd['Risk_StdDev'], risk_return_pd['Avg_Return'], 
            s=100, alpha=0.6, c=range(len(risk_return_pd)), cmap='viridis')

# Thêm label cho một số điểm nổi bật
for idx in range(min(10, len(risk_return_pd))):
    plt.annotate(risk_return_pd.iloc[idx]['Symbol'], 
                (risk_return_pd.iloc[idx]['Risk_StdDev'], 
                 risk_return_pd.iloc[idx]['Avg_Return']),
                fontsize=8, alpha=0.7)

plt.xlabel('Rủi Ro (Độ lệch chuẩn)', fontsize=12)
plt.ylabel('Lợi Nhuận Trung Bình (%)', fontsize=12)
plt.title('BIỂU ĐỒ RỦI RO - LỢI NHUẬN', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='r', linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

print("✅ Đã vẽ biểu đồ rủi ro-lợi nhuận")

# %% [markdown]
# ## 9. Phân tích khối lượng giao dịch

# %%
# Phân tích khối lượng giao dịch theo tháng
df_monthly = df.withColumn("Year", year("Date")) \
    .withColumn("Month", month("Date")) \
    .groupBy("Year", "Month") \
    .agg(
        sum("Volume").alias("Total_Volume"),
        avg("Close").alias("Avg_Price"),
        count("*").alias("Trading_Days")
    ) \
    .orderBy("Year", "Month")

print("\n📊 PHÂN TÍCH THEO THÁNG")
print("=" * 80)
df_monthly.show(24)

# %% [markdown]
# ## 10. Lưu kết quả phân tích

# %%
# Lưu kết quả phân tích về HDFS
output_path = "hdfs://namenode:9000/analysis_results"

print("\n💾 ĐANG LƯU KẾT QUẢ PHÂN TÍCH...")
print("=" * 80)

# Lưu dữ liệu đã phân tích
df_analysis.write.mode("overwrite").parquet(f"{output_path}/stock_analysis")
print("✅ Đã lưu dữ liệu phân tích chi tiết")

# Lưu phân tích theo năm
df_yearly.write.mode("overwrite").parquet(f"{output_path}/yearly_analysis")
print("✅ Đã lưu phân tích theo năm")

# Lưu phân tích rủi ro-lợi nhuận
risk_return.write.mode("overwrite").parquet(f"{output_path}/risk_return_analysis")
print("✅ Đã lưu phân tích rủi ro-lợi nhuận")

print("\n🎉 HOÀN THÀNH PHÂN TÍCH!")

# %% [markdown]
# ## 11. Kết luận
# 
# ### Tóm tắt kết quả:
# 
# 1. **Hệ thống Big Data**: Đã triển khai thành công hệ thống phân tích dữ liệu lớn với Hadoop HDFS và Apache Spark
# 
# 2. **Xử lý dữ liệu**: Xử lý hàng triệu dòng dữ liệu chứng khoán từ nhiều công ty
# 
# 3. **Phân tích**: Thực hiện phân tích xu hướng, tương quan, rủi ro và lợi nhuận
# 
# 4. **Trực quan hóa**: Tạo các biểu đồ trực quan để hỗ trợ ra quyết định đầu tư
# 
# ### Ứng dụng thực tế:
# - Giúp nhà đầu tư đưa ra quyết định thông minh
# - Phát hiện xu hướng và mẫu hình trong thị trường
# - Đánh giá rủi ro và cơ hội đầu tư
# - Tối ưu hóa danh mục đầu tư

# %%
# Dừng Spark Session
spark.stop()
print("\n✅ Đã dừng Spark Session")